In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import matplotlib.pyplot as plt

# Step #1: Load Data
imageFolder = 'maskCheckData'
imds = tf.keras.preprocessing.image_dataset_from_directory(
    imageFolder, labels='inferred', label_mode='categorical', class_names=None, 
    color_mode='rgb', batch_size=32, image_size=(224, 224), shuffle=True, seed=None, 
    validation_split=0.3, subset='training', interpolation='bilinear', follow_links=False
)

# Step #2: Show Loaded Data
tbl = imds.class_names
print(tbl)

# Step #3: Process and Verify Data
imds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    imageFolder, validation_split=0.3, subset='validation', seed=123, image_size=(224, 224)
)

# Display a random image
sample_image_batch, sample_label_batch = next(iter(imds))
plt.imshow(sample_image_batch[0].numpy().astype("uint8"))
plt.title(imds.class_names[sample_label_batch[0].numpy().argmax()])
plt.axis("off")

# Step #4: Import Network and Resize Data
base_model = tf.keras.applications.GoogLeNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Display an example image and its prediction
image_batch, label_batch = next(iter(imds_validation))
image_array = tf.keras.preprocessing.image.img_to_array(random.choice(image_batch))
predicted_label = base_model.predict(np.expand_dims(image_array, axis=0))
predicted_class = tbl[np.argmax(predicted_label[0])]
plt.imshow(tf.keras.preprocessing.image.array_to_img(image_array))
plt.title(predicted_class)
plt.axis("off")

# Step #5: Transfer Learning
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(tbl), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit(imds, validation_data=imds_validation, epochs=6)

# Step #6: Classify on New Data
YPred = model.predict(imds_validation)
accuracy = np.mean(np.argmax(YPred, axis=1) == np.argmax(label_batch, axis=1))
print(f"Accuracy: {accuracy:.4f}")

# Display individual images
for i in range(4):
    image_array = tf.keras.preprocessing.image.img_to_array(random.choice(image_batch))
    predicted_label = model.predict(np.expand_dims(image_array, axis=0))
    predicted_class = tbl[np.argmax(predicted_label[0])]
    plt.figure(figsize=(5, 5))
    plt.imshow(tf.keras.preprocessing.image.array_to_img(image_array))
    plt.title(predicted_class)
    plt.axis("off")


ModuleNotFoundError: No module named 'tensorflow'